### Import Libraries

In [1]:
import numpy as np
import os
import pickle
import csv
import math
from gensim.models import Word2Vec
from sklearn.utils import resample
import random

random.seed(3919)

In [2]:
import pandas as pd
import re
from collections import Counter

In [ ]:
from pprint import pprint
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.test.utils 
import tqdm
import itertools
from sklearn.utils import resample

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pyvis.network import Network

### import dataset

In [3]:
df = pd.read_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/merged_df.pkl')

In [4]:
all_text = " ".join(df['text_clean'])
word_count = len(re.findall(r'\w+', all_text))
print("Total number of words:", word_count)

Total number of words: 8265302


adjust compund words that will be used later for the codebook

In [5]:
word_to_find = 'миропорядок'
mask = df['text_clean'].str.contains(word_to_find)

if mask.any():
    print(f"The word '{word_to_find}' is present in at least one row.")
    rows_with_word = df[mask]
    print(f"The word appears in {len(rows_with_word)} rows.")
else:
    print(f"The word '{word_to_find}' is not present in any row.")

The word 'миропорядок' is present in at least one row.
The word appears in 121 rows.


In [6]:
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("мировой порядок", "миропорядок") 
                if any(phrase in x for phrase in ["мировой порядок", "мирового порядка", 
                                                  "мировому порядку", "мировым порядком",
                                                  "мировом порядке"]) else x)


In [7]:
word_to_find = 'миропорядок'
mask = df['text_clean'].str.contains(word_to_find)

if mask.any():
    print(f"The word '{word_to_find}' is present in at least one row.")
    rows_with_word = df[mask]
    print(f"The word appears in {len(rows_with_word)} rows.")
else:
    print(f"The word '{word_to_find}' is not present in any row.")

The word 'миропорядок' is present in at least one row.
The word appears in 131 rows.


In [8]:
word_to_find = 'русский мир'
mask = df['text_clean'].str.contains(word_to_find)

if mask.any():
    print(f"The word '{word_to_find}' is present in at least one row.")
    rows_with_word = df[mask]
    print(f"The word appears in {len(rows_with_word)} rows.")
else:
    print(f"The word '{word_to_find}' is not present in any row.")

The word 'русский мир' is present in at least one row.
The word appears in 65 rows.


In [ ]:
# transform all declension of русский мир in compound word with _
# nom: русский мир
# gen: русского мира
# dat: русскому миру
# acc: русский мир
# instr: русским миром
# prepos: русском мире

In [9]:
# if русский мир (in all its declension) is present then русский_мир
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("русский мир", "русский_мир"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("русского мира", "русский_мир"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("русскому миру", "русский_мир"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("русским миром", "русский_мир"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("русском мире", "русский_мир"))

In [10]:
word_to_find = 'русский_мир'
mask = df['text_clean'].str.contains(word_to_find)

if mask.any():
    print(f"The word '{word_to_find}' is present in at least one row.")
    rows_with_word = df[mask]
    print(f"The word appears in {len(rows_with_word)} rows.")
else:
    print(f"The word '{word_to_find}' is not present in any row.")

The word 'русский_мир' is present in at least one row.
The word appears in 130 rows.


In [11]:
word_to_find = 'национальная безопасность'
mask = df['text_clean'].str.contains(word_to_find)

if mask.any():
    print(f"The word '{word_to_find}' is present in at least one row.")
    rows_with_word = df[mask]
    print(f"The word appears in {len(rows_with_word)} rows.")
else:
    print(f"The word '{word_to_find}' is not present in any row.")

The word 'национальная безопасность' is present in at least one row.
The word appears in 10 rows.


In [12]:
# the same for национальная_безопасность
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("национальная безопасность", "национальная_безопасность"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("национальной безопасности", "национальная_безопасность"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("национальную безопасность", "национальная_безопасность"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("национальной безопасностью", "национальная_безопасность"))

In [13]:
word_to_find = 'национальная_безопасность'
mask = df['text_clean'].str.contains(word_to_find)

if mask.any():
    print(f"The word '{word_to_find}' is present in at least one row.")
    rows_with_word = df[mask]
    print(f"The word appears in {len(rows_with_word)} rows.")
else:
    print(f"The word '{word_to_find}' is not present in any row.")

The word 'национальная_безопасность' is present in at least one row.
The word appears in 246 rows.


In [14]:
# same for международное право
word_to_find = 'международное право'
mask = df['text_clean'].str.contains(word_to_find)

if mask.any():
    print(f"The word '{word_to_find}' is present in at least one row.")
    rows_with_word = df[mask]
    print(f"The word appears in {len(rows_with_word)} rows.")
else:
    print(f"The word '{word_to_find}' is not present in any row.")

The word 'международное право' is present in at least one row.
The word appears in 414 rows.


In [15]:
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("международное право", "международное_право"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("международного права", "международное_право"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("международному праву", "международное_право"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("международным правом", "международное_право"))
df['text_clean'] = df['text_clean'].apply(lambda x: x.replace("международном праве", "международное_право"))

In [16]:
word_to_find = 'международное_право'
mask = df['text_clean'].str.contains(word_to_find)

if mask.any():
    print(f"The word '{word_to_find}' is present in at least one row.")
    rows_with_word = df[mask]
    print(f"The word appears in {len(rows_with_word)} rows.")
else:
    print(f"The word '{word_to_find}' is not present in any row.")

The word 'международное_право' is present in at least one row.
The word appears in 1554 rows.


### chunk the df into 4 time frames according to presidential mandate

In [17]:
#trasnform date into datetime
df['date'] = pd.to_datetime(df.date)
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df['date'].sort_values()

0      2003-10-06
1      2003-11-11
2      2003-11-13
3      2003-12-03
4      2004-01-12
          ...    
689    2022-12-21
688    2022-12-21
113    2023-01-13
690    2023-02-21
11     2023-02-21
Name: date, Length: 6779, dtype: object

In [18]:
# create boolean masks for each date range
mask1 = (df['date'] >= '2003-01-01') & (df['date'] <= '2008-03-02')
mask2 = (df['date'] >= '2008-03-03') & (df['date'] <= '2012-03-04')
mask3 = (df['date'] >= '2012-03-05') & (df['date'] <= '2018-03-18')
mask4 = (df['date'] >= '2018-03-19') & (df['date'] <= '2023-12-31')

In [19]:
# create smaller dataframes using boolean indexing
t1 = df.loc[mask1] # second Putin's mandate (plus_ a couple of doc from 2003 in Lavrov's):2004-2008
t2 = df.loc[mask2] # Medvedev: 2008-2012
t3 = df.loc[mask3] # third Putin's mandate: 2012-2018
t4 = df.loc[mask4] # fourth Putin's mandate: 2018 - till present (2023)

In [25]:
# Save the four dataframes in separate pickle files
t1.to_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/t1.pkl')
t2.to_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/t2.pkl')
t3.to_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/t3.pkl')
t4.to_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/t4.pkl')

### random sample of 10% for each era

In [9]:
t1 = pd.read_pickle('/Users/adrianacupp/Documents/GitHub/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/t1.pkl')
t2 = pd.read_pickle('/Users/adrianacupp/Documents/GitHub/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/t2.pkl')
t3 = pd.read_pickle('/Users/adrianacupp/Documents/GitHub/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/t3.pkl')
t4 = pd.read_pickle('/Users/adrianacupp/Documents/GitHub/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/t4.pkl')


In [12]:
sample_t1 = t1.sample(frac=0.1, replace=False)
sample_t2 = t2.sample(frac=0.1, replace=False)
sample_t3 = t3.sample(frac=0.1, replace=False)
sample_t4 = t4.sample(frac=0.1, replace=False)

### Codebook

use stem (words that starts as...)

In [10]:
eurasianism = ['еврази','евразийск','русский_мир', 'цивилизаци', 'запад', 'нацисм']

westphalianism = ['великодержавн','статус','суверенн','международное_право']

In [11]:
security_concerns = ['сфер', 'национальная_безопасность','противовес','дестабилиз','нато','угроз']

multipolarism = ['многополярн', 'партнерств', 'сотрудничеств','альтернатив', 'выбор', 'развити']

### transform data in list of lists

In [13]:
# 10% SAMPLE
#transform each timeframe df in a list
era1 = sample_t1['text_clean'].tolist()
era2 = sample_t2['text_clean'].tolist()
era3 = sample_t3['text_clean'].tolist()
era4 = sample_t4['text_clean'].tolist()

In [14]:
list_of_lists = []
list_of_lists.append(era1)
list_of_lists.append(era2)
list_of_lists.append(era3)
list_of_lists.append(era4)

In [15]:
word_freqs = [Counter(sent.split()) for sent in era4]
# print the top 10 most common words in the first document
print(word_freqs[0].most_common(10))

[('и', 34), ('в', 13), ('на', 8), ('с', 7), ('все', 5), ('за', 5), ('медицинских', 4), ('для', 4), ('всего', 4), ('здравоохранения', 4)]


### Iterating over lists of sentences by era to do era-by-era word2vec modeling

In [ ]:
# bootstrapping see rodman
# model: sg
# vector_size: 300
# n_neighbours: 30

In [16]:
# Define function for bootstrapping
# used to generate new samples for training the Word2Vec model
def bootstrap_eras(list_of_lists, n_bootstraps=50, vector_size=100, min_count=0, epochs=200, sg=1, hs=0, negative=5, window=15, workers=4):
    random.seed(3919)
    bootstrapped_eras = []
    for era_idx, era in enumerate(list_of_lists):
        sim_stats_era = []
        for i in range(n_bootstraps):
            sentence_samples = resample(era)
            model = Word2Vec(sentence_samples, vector_size=vector_size, min_count=min_count, epochs=epochs, sg=sg, hs=hs, negative=negative, window=window, workers=workers)
            sim_stats_era.append(model)
            # Save each bootstrapped model as a .pkl file
            filename = f"models/bootstrap/era{era_idx+1}_bootstrap{i+1}.pkl"
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename, "wb") as f:
                pickle.dump(model, f)
            print(f"Finished with run {i+1} out of {n_bootstraps} for era {era_idx+1}")
        bootstrapped_eras.append(sim_stats_era)
    return bootstrapped_eras

In [17]:
bootstrap_eras(list_of_lists, n_bootstraps=50, vector_size=100, min_count=0, epochs=200, sg=1, hs=0, negative=5, window=15, workers=4)

Finished with run 1 out of 50 for era 1
Finished with run 2 out of 50 for era 1
Finished with run 3 out of 50 for era 1
Finished with run 4 out of 50 for era 1
Finished with run 5 out of 50 for era 1
Finished with run 6 out of 50 for era 1
Finished with run 7 out of 50 for era 1
Finished with run 8 out of 50 for era 1
Finished with run 9 out of 50 for era 1
Finished with run 10 out of 50 for era 1
Finished with run 11 out of 50 for era 1
Finished with run 12 out of 50 for era 1
Finished with run 13 out of 50 for era 1
Finished with run 14 out of 50 for era 1
Finished with run 15 out of 50 for era 1
Finished with run 16 out of 50 for era 1
Finished with run 17 out of 50 for era 1
Finished with run 18 out of 50 for era 1
Finished with run 19 out of 50 for era 1
Finished with run 20 out of 50 for era 1
Finished with run 21 out of 50 for era 1
Finished with run 22 out of 50 for era 1
Finished with run 23 out of 50 for era 1
Finished with run 24 out of 50 for era 1
Finished with run 25 out 

In [ ]:
def compute_similarity(bootstrapped_eras, eurasianism, westphalianism, security_concerns, multipolarism):
    similarity = []
    for era in bootstrapped_eras:
        sim_stats_eurasia = []
        sim_stats_westph = []
        sim_stats_sc = []
        sim_stats_mp = []
        for model in era:
            for word, sim_stats in zip(eurasianism, sim_stats_eurasia):
                try:
                    sim_stats.append(model.wv.most_similar('миропорядок', word)[0][1])
                except KeyError:
                    sim_stats.append(np.nan)
            for word, sim_stats in zip(westphalianism, sim_stats_westph):
                try:
                    sim_stats.append(model.wv.most_similar('миропорядок', word)[0][1])
                except KeyError:
                    sim_stats.append(np.nan)
            for word, sim_stats in zip(security_concerns, sim_stats_sc):
                try:
                    sim_stats.append(model.wv.most_similar('миропорядок', word)[0][1])
                except KeyError:
                    sim_stats.append(np.nan)
            for word, sim_stats in zip(multipolarism, sim_stats_mp):
                try:
                    sim_stats.append(model.wv.most_similar('миропорядок', word)[0][1])
                except KeyError:
                    sim_stats.append(np.nan)
        similarity.append([sim_stats_eurasia, sim_stats_westph, sim_stats_sc, sim_stats_mp])
    return similarity

In [ ]:
## Saving model output

with open('naive_model_output.pkl', 'wb') as f:
    pickle.dump(stat_types, f)

then run again the loop but this time just for the interested word

get the cosine similarity between the matrix of entire corpus and the matrix of the single word

get the mean of the bootstrap sample for vectors of sigle word and for vectors of all words from the corpus